# generic space

In [2]:
import nni.retiarii.nn.pytorch as nn
import torch
from collections import OrderedDict

from nni import trace
from nni.retiarii import model_wrapper


In [3]:
@model_wrapper
class SimpleAutoencoderSpace(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        out_layers = OrderedDict(
            [
            ("RelU", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.ReLU())),
            
            ("Sigmoid", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.Sigmoid())),
            ("SiLU",  nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.SiLU())),
            ]
        )

        self.out = nn.LayerChoice(out_layers,label='out')


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.out(x)
        return x

# multi-trial search

In [1]:
from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.strategy import DARTS, Random

from darts.eval_lightning import LightningEval
from darts.phantom import generate_phantom

c:\Users\Public\Public_envs\pub_ml_env\Lib\site-packages\lightning\fabric\__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
c:\Users\Public\Public_envs\pub_ml_env\Lib\site-packages\pkg_resources\__init__.py:2350: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
c:\Users\Public\Public_envs\pub_ml_env\Lib\site-packages\lightning\pytorch\__init__.py:36: DeprecationWarning: Deprecated call to `p

In [4]:
# input image (phantom)
resolution = 7
phantom = generate_phantom(resolution=resolution)

# search space
model_space = SimpleAutoencoderSpace()

evaluator = LightningEval(
                model_space,
                buffer_size=100,
                num_iter=50,
                lr=.0005,
                noise_type='gaussian',
                noise_factor=0.15,
                resolution=resolution,
                )

# search strategy
search_strategy = Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

[2023-08-17 21:04:35] Creating experiment, Experiment ID: hvf5x6mo
[2023-08-17 21:04:35] Starting web server...
[2023-08-17 21:04:36] Setting up...
[2023-08-17 21:04:36] Web portal URLs: http://169.254.138.100:8081 http://169.254.67.161:8081 http://169.254.50.13:8081 http://192.168.0.15:8081 http://127.0.0.1:8081
[2023-08-17 21:04:36] Dispatcher started


AttributeError: 'LightningEval' object has no attribute '_dump'

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()

# one-shot

In [3]:
import torch

from nni import trace
from nni.retiarii import model_wrapper

import nni.retiarii.nn.pytorch as nn

from collections import OrderedDict
from nni.experiment import Experiment
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.strategy import DARTS

from darts.eval_lightning import LightningEval
from darts.phantom import generate_phantom


In [4]:
@model_wrapper
class SimpleAutoencoderSpace(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),
        )

        out_layers = OrderedDict(
            [
            ("RelU", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.ReLU())),
            
            ("Sigmoid", nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.Sigmoid())),
            ("SiLU",  nn.Sequential(
                nn.ConvTranspose2d(1, 1, 3, stride=2, padding=1, output_padding=1),
                nn.SiLU())),
            ]
        )

        self.out = nn.LayerChoice(out_layers,label='out')


    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.out(x)
        return x

In [5]:
# input image (phantom)
resolution = 7
phantom = generate_phantom(resolution=resolution)

# search space
model_space = SimpleAutoencoderSpace()

# model
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=1, out_channels=1, init_features=64, pretrained=False)


# eval module
module = LightningEval(
                model_space, # not sure if space or model goes here
                phantom=phantom, 
                buffer_size=100,
                num_iter=50,
                lr=0.00005, 
                noise_type='gaussian', 
                noise_factor=0.15, 
                resolution=resolution, 
                )
evaluator = FunctionalEvaluator(module)

# search strategy
strategy = DARTS()

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 12   # spawn 50 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1 # will run 1 trial(s) concurrently
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)


exported_arch = experiment.export_top_models(formatter='dict')[0:10]
exported_arch[0]

Using cache found in C:\Users\thesis/.cache\torch\hub\mateuszbuda_brain-segmentation-pytorch_master


[2023-08-17 18:03:14] Creating experiment, Experiment ID: kdp4nxmt
[2023-08-17 18:03:14] Starting web server...
[2023-08-17 18:03:15] Setting up...
[2023-08-17 18:03:15] Web portal URLs: http://169.254.138.100:8081 http://169.254.67.161:8081 http://169.254.50.13:8081 http://10.0.0.172:8081 http://127.0.0.1:8081
[2023-08-17 18:03:15] Dispatcher started


PayloadTooLarge: Pickle too large when trying to dump LightningEval(
  (net): SimpleAutoencoderSpace(
    (encoder): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): ReLU()
    )
    (decoder): Sequential(
      (0): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (1): ReLU()
      (2): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (3): ReLU()
      (4): ConvTranspose2d(16, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (5): Sigmoid()
    )
    (out): LayerChoice(OrderedDict([('RelU', Sequential(
      (0): ConvTranspose2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
 

In [6]:
experiment = Experiment.connect(8081)
experiment.stop()

[2023-08-17 18:03:33] Connect to port 8081 success, experiment id is kdp4nxmt, status is RUNNING.
[2023-08-17 18:03:33] Stopping experiment, please wait...
[2023-08-17 18:03:33] Experiment stopped


[2023-08-17 18:03:33] Dispatcher exiting...
[2023-08-17 18:03:36] Dispatcher terminiated
